In [3]:
import pandas as pd
import numpy as np

## Download stock monthly return data (MSF) over the period of 1990- 2010 from WRDS and complete the following tasks:
- Clean data: take absolute value of price; remove price<$5
- Compute the size (in $1000) of the stocks (price*shrout)
- Get the year and month for each date
- For each stock, get its month t+1 stock return
- Each month, rank all stocks into 10 portfolios by month t return
- Get the equal weighted and market capital weighted average returns at month t+1 for each of the 10 portfolios
- Get the average returns of the 10 portfolios over the entire time period.
- Get the standard deviation of the returns of each portfolio

In [5]:
#Clean data: take absolute value of price; remove price< 5−𝐶𝑜𝑚𝑝𝑢𝑡𝑒𝑡ℎ𝑒𝑠𝑖𝑧𝑒(𝑖𝑛 1000) of the stocks (price*shrout)
def float_data(x):
    try:
        return float(x)
    except:
        return np.nan

def clean_data(data):
    df = data.copy()
    df = df[df.EXCHCD.isin([1,2,3,4])]
    # keeping only the exchange codes 1,2,3,4 for NYSE, NYSE, NASDAQ and ARCA respectively
    df['PRC'] = df['PRC'].apply(lambda x:float_data(x))
    df['SHROUT'] = df['SHROUT'].apply(lambda x:float_data(x))
    df = df[df.PRC >= 5]
    df['size'] = round(df.PRC * df.SHROUT / 1000)
    return df

def month_year(data):
    df = data.copy()
    df['yyyymm'] = df['date'].apply(lambda x:round(x/100))
    return df

def next_month_return(data):
    df = data.copy()
    df['RET'] = df['RET'].apply(lambda x:float_data(x))
    df['RET_1'] = df.groupby('PERMNO')['RET'].shift(-1)
    return df.dropna()

def rank_port(data,var,number):
    df = data.copy()
    df['rank'] = df.groupby("yyyymm")[var].apply(lambda x:pd.qcut(x,number,
                                                                  labels = False,duplicates='drop'))
    # pd.qcut: rank and put variables in equal size bins
    # When number=k, var=x
    # pd.cut: put variables in k bins, each interval length of bin=(max(x)-min(x))/k
    # pd.qcut: put variables in k bins, the percentiles of x: x_i=(i/k)*100 (%)
    # then intervals: (min(x), x_1],(x_1, x_2], (x_2, x_3], ...,(x_{k-1},x_k].
    return df

#value weighted returns
def vw_return(data,var):
    result = data.groupby('yyyymm').apply(lambda x: pd.Series(
                                                           np.average(x[[var]], 
                                                           weights=x["size"], 
                                                           axis=0), [var]))
    return result

#Equal weighted returns
def ew_return(data,var):
    result = data.groupby('yyyymm')[[var]].mean()
    return result

def returns_table(data,var,number,ew=True):
    result = pd.DataFrame()
    if ew:
        result['0'] = ew_return(data[data['rank'] == 0],var = var)
        for n in range(1,number):
            result['{}'.format(n)] = ew_return(data[data['rank'] == n],var = var)
    else:
        result['0'] = vw_return(data[data['rank'] == 0],var = var)
        for n in range(1,number):
            result['{}'.format(n)] = vw_return(data[data['rank'] == n],var = var)
    return result

In [10]:
df = pd.read_csv("monthly_2009to2020.csv")
data = clean_data(data = df)
data_YM = month_year(data)
data_YM.head()

,PERMNO,date,EXCHCD,TICKER,COMNAM,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,SHROUT,size,yyyymm
0,10001,20070131,3.0,EWST,ENERGY WEST INC,11.10,-11.525,11.3584,257.0,0.023279,11.20,11.39,2959.0,34.0,200701
1,10001,20070228,3.0,EWST,ENERGY WEST INC,11.20,14.190,14.1900,1990.0,0.261621,14.01,14.18,2989.0,42.0,200702
2,10001,20070330,3.0,EWST,ENERGY WEST INC,13.81,14.870,14.4700,992.0,0.019732,14.41,14.49,3002.0,43.0,200703
3,10001,20070430,3.0,EWST,ENERGY WEST INC,13.74,15.050,14.5100,1017.0,0.002764,14.51,14.79,3002.0,44.0,200704
4,10001,20070531,3.0,EWST,ENERGY WEST INC,14.23,15.200,14.9900,1155.0,0.043418,14.90,15.00,3003.0,45.0,200705


In [11]:
data_YM_addNextRet = next_month_return(data_YM)
data_YM_addNextRet.head()

,PERMNO,date,EXCHCD,TICKER,COMNAM,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,SHROUT,size,yyyymm,RET_1
0,10001,20070131,3.0,EWST,ENERGY WEST INC,11.10,-11.525,11.3584,257.0,0.023279,11.20,11.39,2959.0,34.0,200701,0.261621
1,10001,20070228,3.0,EWST,ENERGY WEST INC,11.20,14.190,14.1900,1990.0,0.261621,14.01,14.18,2989.0,42.0,200702,0.019732
2,10001,20070330,3.0,EWST,ENERGY WEST INC,13.81,14.870,14.4700,992.0,0.019732,14.41,14.49,3002.0,43.0,200703,0.002764
3,10001,20070430,3.0,EWST,ENERGY WEST INC,13.74,15.050,14.5100,1017.0,0.002764,14.51,14.79,3002.0,44.0,200704,0.043418
4,10001,20070531,3.0,EWST,ENERGY WEST INC,14.23,15.200,14.9900,1155.0,0.043418,14.90,15.00,3003.0,45.0,200705,0.000667


In [12]:
x = rank_port(data_YM_addNextRet,var='RET',number = 10)
x

,PERMNO,date,EXCHCD,TICKER,COMNAM,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,SHROUT,size,yyyymm,RET_1,rank
0,10001,20070131,3.0,EWST,ENERGY WEST INC,11.10000,-11.52500,11.35840,257.0,0.023279,11.20000,11.39000,2959.0,34.0,200701,0.261621,5
1,10001,20070228,3.0,EWST,ENERGY WEST INC,11.20000,14.19000,14.19000,1990.0,0.261621,14.01000,14.18000,2989.0,42.0,200702,0.019732,9
2,10001,20070330,3.0,EWST,ENERGY WEST INC,13.81000,14.87000,14.47000,992.0,0.019732,14.41000,14.49000,3002.0,43.0,200703,0.002764,6
3,10001,20070430,3.0,EWST,ENERGY WEST INC,13.74000,15.05000,14.51000,1017.0,0.002764,14.51000,14.79000,3002.0,44.0,200704,0.043418,3
4,10001,20070531,3.0,EWST,ENERGY WEST INC,14.23000,15.20000,14.99000,1155.0,0.043418,14.90000,15.00000,3003.0,45.0,200705,0.000667,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204189,93436,20200731,3.0,TSLA,TESLA INC,1119.63000,1643.00000,1430.76001,3776740.0,0.325011,1431.89001,1432.00000,186362.0,266639.0,202007,0.741452,9
1204190,93436,20200831,3.0,TSLA,TESLA INC,498.32001,2238.75000,498.32001,4051970.0,0.741452,498.28000,498.51001,931809.0,464339.0,202008,-0.139087,9
1204191,93436,20200930,3.0,TSLA,TESLA INC,330.20999,475.04999,429.01001,17331954.0,-0.139087,428.82001,429.01999,948000.0,406701.0,202009,-0.095499,0
1204192,93436,20201030,3.0,TSLA,TESLA INC,388.04001,461.29999,388.04001,8330610.0,-0.095499,388.04001,388.35001,947901.0,367824.0,202010,0.462736,0


In [13]:
#ew
df_ew = returns_table(data = x, var = 'RET_1',number = 10)
#vw
df_vw = returns_table(data = x, var = 'RET_1',number = 10,ew=False)

In [14]:
display(df_ew.describe().iloc[1:3])

,0,1,2,3,4,5,6,7,8,9
mean,0.024098,0.014074,0.011852,0.010141,0.009799,0.009294,0.009511,0.009546,0.009884,0.015316
std,0.062032,0.052858,0.047702,0.043913,0.042516,0.041821,0.040997,0.040845,0.042351,0.047886


In [15]:
display(df_vw.describe().iloc[1:3])

,0,1,2,3,4,5,6,7,8,9
mean,0.008845,0.009566,0.008661,0.009922,0.007634,0.009294,0.009379,0.007536,0.008075,0.008959
std,0.070089,0.057921,0.050077,0.046571,0.044106,0.044829,0.043067,0.043495,0.045831,0.052329
